https://pub.towardsai.net/connect-colab-to-gcs-bucket-using-gcsfuse-29f4f844d074


In [ ]:
# https://stackoverflow.com/questions/71683158/authentication-issues-on-colab-bigquery
import os
os.environ['USE_AUTH_EPHEM'] = '0'

from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  30250      0 --:--:-- --:--:-- --:--:-- 31025
OK
72 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.42.4).
0 upgraded, 0 newly installed, 0 to remove and 72 not upgraded.


In [ ]:
!mkdir mura
!gcsfuse --implicit-dirs mura_data_606 mura

mkdir: cannot create directory ‘mura’: File exists
I0514 07:26:09.492302 2023/05/14 07:26:09.492273 Start gcsfuse/0.42.4 (Go version go1.19.7) for app "" using mount point: /content/mura


In [ ]:
%%capture
! pip install datasets transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
from torchvision.io import read_image
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm
from timeit import default_timer as timer
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer, ViTFeatureExtractor, ViTImageProcessor
from datasets import load_metric, Dataset
from transformers import TrainingArguments, Trainer, AutoModelForImageClassification, AutoImageProcessor, pipeline
import copy
import time
import torch.optim as optim
import torchvision
from skimage import io

In [ ]:
# must use cuda for ViT
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Load and Prep Data

## Import and Massage Data

In [ ]:
# load train and test csv's with all image paths and targets
train_img_paths = pd.read_csv('mura/train_image_paths.csv', header = None, names = ['image_path'])
test_img_paths = pd.read_csv('mura/valid_image_paths.csv', header = None, names = ['image_path'])

In [ ]:
len(train_img_paths)

36808

In [ ]:
pd.set_option('max_colwidth', None)
train_img_paths.head()

,image_path
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image2.png
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image3.png
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image1.png
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study1_positive/image2.png


In [ ]:
# funtion to extract info from train and test dataframes
def split_img_paths(df):
    temp = df['image_path'].str.split('/', expand=True)
    new_df = pd.DataFrame(df['image_path'])
    new_df['patient_id'] = temp[3]
    new_df['study'] = temp[4].str.split('_', expand = True)[0].str[-1:].astype(int)
    new_df['region'] = temp[2].str.split('_', expand = True)[1]
    map = {'positive':'abnormal', 'negative':'normal'}
    new_df['target'] = temp[4].str.split('_', expand = True)[1]
    new_df['target'] = new_df['target'].map(map)
    #add image path
    new_df['image_path'] = 'mura/'+temp[1]+'/'+temp[2]+'/'+temp[3]+'/'+temp[4]+'/'+temp[5] 
    
    return new_df

In [ ]:
# create df with info extracted
mura_train = split_img_paths(train_img_paths)
mura_test = split_img_paths(test_img_paths)
mura_train.head()

,image_path,patient_id,study,region,target
0,mura/train/XR_SHOULDER/patient00001/study1_positive/image1.png,patient00001,1,SHOULDER,abnormal
1,mura/train/XR_SHOULDER/patient00001/study1_positive/image2.png,patient00001,1,SHOULDER,abnormal
2,mura/train/XR_SHOULDER/patient00001/study1_positive/image3.png,patient00001,1,SHOULDER,abnormal
3,mura/train/XR_SHOULDER/patient00002/study1_positive/image1.png,patient00002,1,SHOULDER,abnormal
4,mura/train/XR_SHOULDER/patient00002/study1_positive/image2.png,patient00002,1,SHOULDER,abnormal


In [ ]:
# function to create df with 14 targets
def mura_to14(df):
    new_df = df.copy()
    new_df['target'] = df['target'] + '_' + df['region']
    return new_df

In [ ]:
# create df with 14 targets
mura_train_14 = mura_to14(mura_train)
mura_test_14 = mura_to14(mura_test)


In [ ]:
mura_train_14.head()

,image_path,patient_id,study,region,target
0,mura/train/XR_SHOULDER/patient00001/study1_positive/image1.png,patient00001,1,SHOULDER,abnormal_SHOULDER
1,mura/train/XR_SHOULDER/patient00001/study1_positive/image2.png,patient00001,1,SHOULDER,abnormal_SHOULDER
2,mura/train/XR_SHOULDER/patient00001/study1_positive/image3.png,patient00001,1,SHOULDER,abnormal_SHOULDER
3,mura/train/XR_SHOULDER/patient00002/study1_positive/image1.png,patient00002,1,SHOULDER,abnormal_SHOULDER
4,mura/train/XR_SHOULDER/patient00002/study1_positive/image2.png,patient00002,1,SHOULDER,abnormal_SHOULDER


In [ ]:
# create train df by region
wrist_train = mura_train[mura_train['region']=='WRIST']
forearm_train = mura_train[mura_train['region']=='FOREARM']
hand_train = mura_train[mura_train['region']=='HAND']
humerus_train = mura_train[mura_train['region']=='HUMERUS']
elbow_train = mura_train[mura_train['region']=='ELBOW']
shoulder_train = mura_train[mura_train['region']=='SHOULDER']
finger_train = mura_train[mura_train['region']=='FINGER']

# create test df by region
wrist_test = mura_test[mura_test['region']=='WRIST']
forearm_test = mura_test[mura_test['region']=='FOREARM']
hand_test = mura_test[mura_test['region']=='HAND']
humerus_test = mura_test[mura_test['region']=='HUMERUS']
elbow_test = mura_test[mura_test['region']=='ELBOW']
shoulder_test = mura_test[mura_test['region']=='SHOULDER']
finger_test = mura_test[mura_test['region']=='FINGER']

In [ ]:
wrist_test.head()

,image_path,patient_id,study,region,target
0,mura/valid/XR_WRIST/patient11185/study1_positive/image1.png,patient11185,1,WRIST,abnormal
1,mura/valid/XR_WRIST/patient11185/study1_positive/image2.png,patient11185,1,WRIST,abnormal
2,mura/valid/XR_WRIST/patient11185/study1_positive/image3.png,patient11185,1,WRIST,abnormal
3,mura/valid/XR_WRIST/patient11185/study1_positive/image4.png,patient11185,1,WRIST,abnormal
4,mura/valid/XR_WRIST/patient11186/study1_positive/image1.png,patient11186,1,WRIST,abnormal


## Create custom Datasets and Dataloaders

https://www.learnpytorch.io/04_pytorch_custom_datasets#41-turn-loaded-images-into-dataloaders

In [ ]:
# transformations for images
data_transforms = transforms.Compose([
    #
    #note that output channel is 1, will need to change and re-run for use with 3 channels
    #
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
# function to provide list of classes and a dict of classes:targets
def class_dict(two_targets):
    classes_2 = ['normal', 'abnormal']
    dict_2 = {'normal' : 0, 'abnormal': 1}
    classes_14 = ['abnormal_WRIST', 'normal_WRIST','abnormal_FOREARM', 'normal_FOREARM',
                      'abnormal_HAND', 'normal_HAND', 'abnormal_HUMERUS', 'normal_HUMERUS',
                      'abnormal_ELBOW', 'normal_ELBOW', 'abnormal_SHOULDER', 'normal_SHOULDER',
                      'abnormal_FINGER', 'normal_FINGER']
    dict_14 = {'abnormal_WRIST': 0, 'normal_WRIST': 1,'abnormal_FOREARM' : 2, 'normal_FOREARM' : 3,
                      'abnormal_HAND': 4, 'normal_HAND' : 5, 'abnormal_HUMERUS': 6, 'normal_HUMERUS' : 7,
                      'abnormal_ELBOW': 8, 'normal_ELBOW' : 9, 'abnormal_SHOULDER' : 10, 'normal_SHOULDER': 11,
                      'abnormal_FINGER': 12, 'normal_FINGER' : 13}
    if two_targets == True:
      return classes_2, dict_2
    else:
      return classes_14, dict_14

In [ ]:
# create custom dataset class for our MURA dataset

class MuraDataset(Dataset):
    # change two targets to False if using 14 target data
    def __init__(self, mura_df, two_targets, transform=data_transforms):
        self.mura = mura_df
        self.transform = transform
        self.classes, self.class_to_idx = class_dict(two_targets=two_targets)

    def __len__(self):
        return len(self.mura)

    # retrives image from path in df file and target from df
    def get_image_class(self, index):
        image_path = self.mura.iloc[index]['image_path']
        class_name = self.mura.iloc[index]['target']
        return Image.open(image_path), class_name  

    def __getitem__(self, idx):
        image, class_name = self.get_image_class(idx)
        label = self.class_to_idx[class_name]

        return self.transform(image), label

In [ ]:
#dict = {}
#for i in range(len(train_dataset)):
#    img, label = next(iter(train_dataloader))
#    shape = img.shape
#    dict[shape] = dict.get(shape, 0) + 1

In [ ]:
# create train and test datasets for 14 target df
mura_train_14_dataset = MuraDataset(mura_df = mura_train_14, two_targets=False)
mura_test_14_dataset = MuraDataset(mura_df = mura_test_14, two_targets=False)

In [ ]:
print(len(mura_train_14_dataset), len(mura_test_14_dataset))

36808 3197


In [ ]:
# create train datasets by region
wrist_train_dataset = MuraDataset(mura_df = wrist_train, two_targets=True)
forearm_train_dataset = MuraDataset(mura_df = forearm_train, two_targets=True)
hand_train_dataset = MuraDataset(mura_df = hand_train, two_targets=True)
humerus_train_dataset = MuraDataset(mura_df = humerus_train, two_targets=True)
elbow_train_dataset = MuraDataset(mura_df = elbow_train, two_targets=True)
shoulder_train_dataset = MuraDataset(mura_df = shoulder_train, two_targets=True)
finger_train_dataset = MuraDataset(mura_df = finger_train, two_targets=True)

# create test datasets by region
wrist_test_dataset = MuraDataset(mura_df = wrist_test, two_targets=True)
forearm_test_dataset = MuraDataset(mura_df = forearm_test, two_targets=True)
hand_test_dataset = MuraDataset(mura_df = hand_test, two_targets=True)
humerus_test_dataset = MuraDataset(mura_df = humerus_test, two_targets=True)
elbow_test_dataset = MuraDataset(mura_df = elbow_test, two_targets=True)
shoulder_test_dataset = MuraDataset(mura_df = shoulder_test, two_targets=True)
finger_test_dataset = MuraDataset(mura_df = finger_test, two_targets=True)

In [ ]:
wrist_len = len(wrist_train)
forearm_len = len(forearm_train)
hand_len = len(hand_train)
humerus_len = len(humerus_train)
elbow_len = len(elbow_train)
shoulder_len = len(shoulder_train)
finger_len = len(finger_train)

total = len(mura_train)

# ratio of portion of samples we want from each region
wrist_ratio = round(wrist_len/total,5)
forearm_ratio = round(forearm_len/total,5)
hand_ratio = round(hand_len/total,5)
humerus_ratio = round(humerus_len/total,5)
elbow_ratio = round(elbow_len/total,5)
shoulder_ratio = round(shoulder_len/total,5)
finger_ratio = round(finger_len/total,5)


In [ ]:
print(wrist_ratio, forearm_ratio)

0.26494 0.04958


In [ ]:
# create subset of 14 target Dataset, returns train and test Dataset 
def mura_sample(two_variables = True, num_train_samples = 7000, train_portion = 0.8, eq_reg_samp = False, vit=False, over_sample=False):
    
    # get number of test samples to sample
    num_test_samples = (1-train_portion) * num_train_samples / train_portion

    # use len of train data to get porportion of data we want to sample from each region
    wrist_len = len(wrist_train)
    forearm_len = len(forearm_train)
    hand_len = len(hand_train)
    humerus_len = len(humerus_train)
    elbow_len = len(elbow_train)
    shoulder_len = len(shoulder_train)
    finger_len = len(finger_train)

    total = len(mura_train)

    # ratio of portion of samples we want from each region
    wrist_ratio = round(wrist_len/total,5) if eq_reg_samp is False else round(1/7,5)
    forearm_ratio = round(forearm_len/total,5) if eq_reg_samp is False else round(1/7,5)
    hand_ratio = round(hand_len/total,5) if eq_reg_samp is False else round(1/7,5)
    humerus_ratio = round(humerus_len/total,5) if eq_reg_samp is False else round(1/7,5)
    elbow_ratio = round(elbow_len/total,5) if eq_reg_samp is False else round(1/7,5)
    shoulder_ratio = round(shoulder_len/total,5) if eq_reg_samp is False else round(1/7,5)
    finger_ratio = round(finger_len/total,5) if eq_reg_samp is False else round(1/7,5)

    # random sample train regions
    a = wrist_train.sample(n=int(wrist_ratio*num_train_samples), replace=over_sample)
    b = forearm_train.sample(n=int(forearm_ratio*num_train_samples), replace=over_sample)
    c = hand_train.sample(n=int(hand_ratio*num_train_samples), replace=over_sample)
    d = humerus_train.sample(n=int(humerus_ratio*num_train_samples), replace=over_sample)
    e = elbow_train.sample(n=int(elbow_ratio*num_train_samples), replace=over_sample)
    f = shoulder_train.sample(n=int(shoulder_ratio*num_train_samples), replace=over_sample)
    g = finger_train.sample(n=int(finger_ratio*num_train_samples), replace=over_sample)

    # random sample test regions
    h = wrist_test.sample(n=int(wrist_ratio*num_test_samples), replace=over_sample)
    i = forearm_test.sample(n=int(forearm_ratio*num_test_samples), replace=over_sample)
    j = hand_test.sample(n=int(hand_ratio*num_test_samples), replace=over_sample)
    k = humerus_test.sample(n=int(humerus_ratio*num_test_samples), replace=over_sample)
    l = elbow_test.sample(n=int(elbow_ratio*num_test_samples), replace=over_sample)
    m = shoulder_test.sample(n=int(shoulder_ratio*num_test_samples), replace=over_sample)
    n = finger_test.sample(n=int(finger_ratio*num_test_samples), replace=over_sample)

    train_frames = [a,b,c,d,e,f,g]
    test_frames = [h,i,j,k,l,m,n]
    keys = mura_train.columns
    
    train_df = pd.concat(train_frames, keys=keys)
    test_df = pd.concat(test_frames, keys=keys)

    train_df_shuff = train_df.sample(frac=1).reset_index(drop=True)
    test_df_shuff = test_df.sample(frac=1).reset_index(drop=True)

    if two_variables is False:
        train_df_shuff = mura_to14(train_df_shuff)
        test_df_shuff = mura_to14(test_df_shuff)

    if vit is False:
        train_dataset = MuraDataset(mura_df = train_df_shuff, two_targets=two_variables)
        test_dataset = MuraDataset(mura_df = test_df_shuff, two_targets=two_variables)
    else:
        train_dataset = train_df_shuff
        test_dataset = test_df_shuff

    return train_dataset, test_dataset

In [ ]:
mura_train_14_subset, mura_test_14_subset = mura_sample(two_variables = False, vit=False)
mura_train_subset, mura_test_subset = mura_sample(two_variables = True, vit=False)

In [ ]:
print(len(mura_train_14_subset), len(mura_test_14_subset))

4433 1106


In [ ]:
print(len(mura_train_subset), len(mura_test_subset))

4433 1106


In [ ]:
#check size of datasets
print(f'wrist: {len(wrist_train_dataset)}, {len(wrist_test_dataset)}')
print(f'forearm: {len(forearm_train_dataset)}, {len(forearm_test_dataset)}')
print(f'hand: {len(hand_train_dataset)}, {len(hand_test_dataset)}')
print(f'humerus: {len(humerus_train_dataset)}, {len(humerus_test_dataset)}')
print(f'elbow: {len(elbow_train_dataset)}, {len(elbow_test_dataset)}')
print(f'shoulder: {len(shoulder_train_dataset)}, {len(shoulder_test_dataset)}')
print(f'finger: {len(finger_train_dataset)}, {len(finger_test_dataset)}')

wrist: 9752, 659
forearm: 1825, 301
hand: 5543, 460
humerus: 1272, 288
elbow: 4931, 465
shoulder: 8379, 563
finger: 5106, 461


In [ ]:
# batch size for all dataloaders we are creating
batch_size = 32

In [ ]:
# create dataloader for 14 targets dataset
mura_train_14_dataloader = DataLoader(mura_train_14_dataset, batch_size=batch_size, shuffle=True)
mura_test_14_dataloader = DataLoader(mura_test_14_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#create train dataloaders by region
wrist_train_dataloader = DataLoader(wrist_train_dataset, batch_size=batch_size, shuffle=True)
forearm_train_dataloader = DataLoader(forearm_train_dataset, batch_size=batch_size, shuffle=True)
hand_train_dataloader = DataLoader(hand_train_dataset, batch_size=batch_size, shuffle=True)
humerus_train_dataloader = DataLoader(humerus_train_dataset, batch_size=batch_size, shuffle=True)
elbow_train_dataloader = DataLoader(elbow_train_dataset, batch_size=batch_size, shuffle=True)
shoulder_train_dataloader = DataLoader(shoulder_train_dataset, batch_size=batch_size, shuffle=True)
finger_train_dataloader = DataLoader(finger_train_dataset, batch_size=batch_size, shuffle=True)

#create test dataloaders by region
wrist_test_dataloader = DataLoader(wrist_test_dataset, batch_size=batch_size, shuffle=True)
forearm_test_dataloader = DataLoader(forearm_test_dataset, batch_size=batch_size, shuffle=True)
hand_test_dataloader = DataLoader(hand_test_dataset, batch_size=batch_size, shuffle=True)
humerus_test_dataloader = DataLoader(humerus_test_dataset, batch_size=batch_size, shuffle=True)
elbow_test_dataloader = DataLoader(elbow_test_dataset, batch_size=batch_size, shuffle=True)
shoulder_test_dataloader = DataLoader(shoulder_test_dataset, batch_size=batch_size, shuffle=True)
finger_test_dataloader = DataLoader(finger_test_dataset, batch_size=batch_size, shuffle=True)

# Base Model

## Model Implementation

Implement CNN from:

https://www.learnpytorch.io/03_pytorch_computer_vision/#7-model-2-building-a-convolutional-neural-network-cnn

In [ ]:
# Create a convolutional neural network 
class base_model(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*56*56, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

In [ ]:
# define how we will measure our model accuracy
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
# define our training and test steps
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")


In [ ]:
# calculate total time training
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time/60:.3f} minutes")
    return total_time

In [ ]:
# Measure time

def run_model(train_data_loader, test_data_loader, model, loss_fn, optimizer,
              accuracy_fn=accuracy_fn, epochs=3, device=device):
    train_time_start = timer()

    # Train and test model 
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n---------")
        train_step(data_loader=train_data_loader, 
            model=model, 
            loss_fn=loss_fn,
            optimizer=optimizer,
            accuracy_fn=accuracy_fn,
            device=device
        )
        test_step(data_loader=test_data_loader,
            model=model,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device
        )

    train_time_end = timer()

    total_train_time = print_train_time(start=train_time_start,
                                              end=train_time_end,
                                              device=device)
    print(total_train_time)

## Run Models

In [ ]:
# define loss function and optimizer for all models
loss_fn_base_model = nn.CrossEntropyLoss()

lr_base_model = 0.01

# set optimizer with different model params
def set_optimizer(model, lr=lr_base_model):
    opt = torch.optim.SGD(params=model.parameters(), 
                             lr=lr)
    return opt 

epochs_base_model = 5
seed = 3

### 14 targets

In [ ]:
# create 14 target base model and move to device
torch.manual_seed(seed)

mura_14_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(mura_train_14_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(mura_14_base_model)

mura_14_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=31360, out_features=14, bias=True)
  )
)

In [ ]:
# train and test 14 target base model
run_model(train_data_loader=mura_train_14_dataloader, test_data_loader=mura_test_14_dataloader,
          model=mura_14_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
---------


  0%|          | 0/5 [00:32<?, ?it/s]


KeyboardInterrupt: ignored

### Wrist

In [ ]:
# create wrist base model and move to device
torch.manual_seed(seed)

wrist_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(wrist_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(wrist_base_model)

wrist_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=31360, out_features=2, bias=True)
  )
)

In [ ]:
# train and test wrist base model
run_model(train_data_loader=wrist_train_dataloader, test_data_loader=wrist_test_dataloader,
          model=wrist_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Forearm

In [ ]:
# create forearm base model and move to device
torch.manual_seed(seed)

forearm_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(forearm_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(forearm_base_model)

forearm_base_model

In [ ]:
# train and test forearm base model
run_model(train_data_loader=forearm_train_dataloader, test_data_loader=forearm_test_dataloader,
          model=forearm_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Hand

In [ ]:
# create hand base model and move to device
torch.manual_seed(seed)

hand_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(hand_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(hand_base_model)

hand_base_model

In [ ]:
# train and test hand base model
run_model(train_data_loader=hand_train_dataloader, test_data_loader=hand_test_dataloader,
          model=hand_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Humerus

In [ ]:
# create humerus base model and move to device
torch.manual_seed(seed)

humerus_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(humerus_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(humerus_base_model)

humerus_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test humerus base model
run_model(train_data_loader=humerus_train_dataloader, test_data_loader=humerus_test_dataloader,
          model=humerus_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Shoulder

In [ ]:
# create shoulder base model and move to device
torch.manual_seed(seed)

shoulder_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(shoulder_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(shoulder_base_model)

shoulder_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test shoulder base model
run_model(train_data_loader=shoulder_train_dataloader, test_data_loader=shoulder_test_dataloader,
          model=shoulder_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

### Elbow

In [ ]:
elbow_base_model.parameters

<bound method Module.parameters of base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)>

In [ ]:
# create wrist base elbow and move to device
torch.manual_seed(3)

elbow_base_model = base_model(input_shape=50176, 
    hidden_units=10, 
    output_shape=len(elbow_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(elbow_base_model)

elbow_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(50176, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test elbow base model
run_model(train_data_loader=elbow_train_dataloader, test_data_loader=elbow_test_dataloader,
          model=elbow_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

Elbow With batch size = 1
 
  0%|          | 0/3 [00:00<?, ?it/s]Epoch: 0
---------
Train loss: 2927377804200474425950208.00000 | Train accuracy: 57.96%
 33%|███▎      | 1/3 [1:32:51<3:05:42, 5571.33s/it]Test loss: 0.68236 | Test accuracy: 59.32%

Epoch: 1
---------
Train loss: 0.68688 | Train accuracy: 57.41%
 67%|██████▋   | 2/3 [1:34:09<39:00, 2340.27s/it]  Test loss: 0.70821 | Test accuracy: 59.32%

Epoch: 2
---------
Train loss: 0.68935 | Train accuracy: 57.70%
100%|██████████| 3/3 [1:35:26<00:00, 1908.91s/it]Test loss: 0.71157 | Test accuracy: 40.68%



Elbow With batch size = 32

0%|          | 0/3 [00:00<?, ?it/s]Epoch: 0
---------
Train loss: 0.67527 | Train accuracy: 59.56%
 33%|███▎      | 1/3 [19:56<39:53, 1196.83s/it]Test loss: 0.67626 | Test accuracy: 59.36%

Epoch: 1
---------
Train loss: 0.67621 | Train accuracy: 59.36%
 67%|██████▋   | 2/3 [21:01<08:50, 530.74s/it] Test loss: 0.67643 | Test accuracy: 59.17%

Epoch: 2
---------
Train loss: 0.67608 | Train accuracy: 59.36%
100%|██████████| 3/3 [22:04<00:00, 441.58s/it]Test loss: 0.67713 | Test accuracy: 58.97%


Train time on cuda: 22.079 minutes


### Finger

In [ ]:
# create finger base model and move to device
torch.manual_seed(seed)

finger_base_model = base_model(input_shape=1, 
    hidden_units=10, 
    output_shape=len(finger_test_dataset.classes)).to(device)

optimizer_base_model = set_optimizer(finger_base_model)

finger_base_model

base_model(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=40960, out_features=2, bias=True)
  )
)

In [ ]:
# train and test finger base model
run_model(train_data_loader=finger_train_dataloader, test_data_loader=finger_test_dataloader,
          model=finger_base_model, loss_fn=loss_fn_base_model, optimizer=optimizer_base_model, accuracy_fn=accuracy_fn,
          epochs=epochs_base_model)

# Vision Transfomer

https://huggingface.co/blog/fine-tune-vit

 https://www.learnpytorch.io/04_pytorch_custom_datasets/#41-turn-loaded-images-into-dataloaders

In [ ]:
#pre_trained model we are using
# Vision Transformer (ViT) model pre-trained on ImageNet-21k 
# (14 million images, 21,843 classes) at resolution 224x224.
model_name_or_path = 'google/vit-base-patch16-224-in21k'

Images are resized/rescaled to the same resolution (224x224) and normalized across the RGB channels with mean (0.5, 0.5, 0.5) and standard deviation (0.5, 0.5, 0.5).

In [ ]:
feature_extractor = ViTImageProcessor.from_pretrained(model_name_or_path)

## Create Custom Datasets

In [ ]:
target_to_label_2 = {'normal' : 0, 'abnormal': 1}
label_to_target_2 = {0 : 'normal', 1 : 'abnormal'}

In [ ]:
target_to_label_14 = {'abnormal_WRIST': 0, 'normal_WRIST': 1,'abnormal_FOREARM' : 2, 'normal_FOREARM' : 3,
                      'abnormal_HAND': 4, 'normal_HAND' : 5, 'abnormal_HUMERUS': 6, 'normal_HUMERUS' : 7,
                      'abnormal_ELBOW': 8, 'normal_ELBOW' : 9, 'abnormal_SHOULDER' : 10, 'normal_SHOULDER': 11,
                      'abnormal_FINGER': 12, 'normal_FINGER' : 13}

label_to_target_14 = {0: 'abnormal_WRIST', 1:'normal_WRIST', 2:'abnormal_FOREARM', 3:'normal_FOREARM',
                      4:'abnormal_HAND', 5:'normal_HAND', 6:'abnormal_HUMERUS', 7:'normal_HUMERUS',
                      8:'abnormal_ELBOW', 9:'normal_ELBOW', 10:'abnormal_SHOULDER', 11:'normal_SHOULDER',
                      12:'abnormal_FINGER', 13:'normal_FINGER'}

In [ ]:
# transform our dataset to match pre trained model
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel value
    images = []
    for img_path in example_batch['image_path']:
        img_read = Image.open(img_path).convert('RGB')
        images.append(img_read)
    inputs = feature_extractor([x for x in images], return_tensors='pt')

    # Don't forget to include the labels!
    targets = []
    for targ in example_batch['target']:
        targets.append(target_to_label_2[targ])
    inputs['labels'] = [x for x in targets]
    return inputs

In [ ]:
# create ViT test and train datasets for 14 targets
ViT_mura_train_14_dataset = Dataset.from_pandas(mura_train_14).with_transform(transform)
ViT_mura_test_14_dataset = Dataset.from_pandas(mura_test_14).with_transform(transform)


ViT_mura_train_all = Dataset.from_pandas(mura_train.sample(frac=1).reset_index(drop=True)).with_transform(transform)
ViT_mura_test_all = Dataset.from_pandas(mura_test.sample(frac=1).reset_index(drop=True)).with_transform(transform)

In [ ]:
# create subset of ViT test and train datasets for 14 targets
#ViT_mura_train_14_subset, ViT_mura_test_14_subset = mura_sample(two_variables = False, vit=True)

ViT_mura_train_subset, ViT_mura_test_subset = mura_sample(vit=True)

ViT_mura_train_subset = Dataset.from_pandas(ViT_mura_train_subset.sample(frac=1).reset_index(drop=True)).with_transform(transform)
ViT_mura_test_subset = Dataset.from_pandas(ViT_mura_test_subset.sample(frac=1).reset_index(drop=True)).with_transform(transform)

In [ ]:
# create ViT train datasets by region
ViT_shoulder_train_dataset = Dataset.from_pandas(shoulder_train).with_transform(transform)
ViT_humerus_train_dataset = Dataset.from_pandas(humerus_train).with_transform(transform)
ViT_finger_train_dataset = Dataset.from_pandas(finger_train).with_transform(transform)
ViT_elbow_train_dataset = Dataset.from_pandas(elbow_train).with_transform(transform)
ViT_wrist_train_dataset = Dataset.from_pandas(wrist_train).with_transform(transform)
ViT_forearm_train_dataset = Dataset.from_pandas(forearm_train).with_transform(transform)
ViT_hand_train_dataset = Dataset.from_pandas(hand_train).with_transform(transform)


# create ViT test datasets by region
ViT_shoulder_test_dataset = Dataset.from_pandas(shoulder_test).with_transform(transform)
ViT_humerus_test_dataset = Dataset.from_pandas(humerus_test).with_transform(transform)
ViT_finger_test_dataset = Dataset.from_pandas(finger_test).with_transform(transform)
ViT_elbow_test_dataset = Dataset.from_pandas(elbow_test).with_transform(transform)
ViT_wrist_test_dataset = Dataset.from_pandas(wrist_test).with_transform(transform)
ViT_forearm_test_dataset = Dataset.from_pandas(forearm_test).with_transform(transform)
ViT_hand_test_dataset = Dataset.from_pandas(hand_test).with_transform(transform)

In [ ]:
val_sets = {'shoulder' : ViT_shoulder_test_dataset, 'humerus' : ViT_humerus_test_dataset, 
            'finger' : ViT_finger_test_dataset, 'elbow' : ViT_elbow_test_dataset, 
            'wrist' : ViT_wrist_test_dataset, 'forearm' : forearm_test_dataset,
            'hand' : ViT_hand_test_dataset}

## Run ViT Models

In [ ]:
# define our collate function
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [ ]:
# define metric same as base model
metric = load_metric("accuracy")
#metric = load_metric("precision")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [ ]:
# training argument for all ViTs
vit_batch_size = 32
vit_epochs = 5
vit_lr = 2e-4

### 14 targets

In [ ]:
training_args_14_targets = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="drive/MyDrive/Colab Notebooks/Training_Checkpoints/14_targets",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for 14 target ViT model
ViT_14_target_trainer = Trainer(
    model=ViT_14_target_model,
    args=training_args_14_targets,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_mura_train_14_dataset,
    eval_dataset=ViT_mura_test_14_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run 14 target ViT model


In [ ]:
print(ViT_14_targets_results)

### 14 targets subset

In [ ]:
training_args_14_targets_subset = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="drive/MyDrive/Colab Notebooks/Training_Checkpoints/14_targets_subset",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for 14 target ViT model
ViT_14_target_sub_trainer = Trainer(
    model=ViT_14_target_model,
    args=training_args_14_targets_subset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_mura_train_14_subset,
    eval_dataset=ViT_mura_test_14_subset,
    tokenizer=feature_extractor,
)

In [ ]:
# run 14 target ViT model


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
print(ViT_14_targets_sub_results)

### All Data Subset

In [ ]:
ViT_mura_sub_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_mura_subset = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura_Models",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for mura subset ViT model
ViT_mura_sub_trainer = Trainer(
    model=ViT_mura_sub_model,
    args=training_args_mura_subset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_mura_train_subset,
    eval_dataset=ViT_mura_test_subset,
    tokenizer=feature_extractor,
)

In [ ]:
# run mura subset ViT model
mura_subset_train_results = ViT_mura_sub_trainer.train()
ViT_mura_sub_trainer.save_model()
ViT_mura_sub_trainer.log_metrics("train", mura_subset_train_results.metrics)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.628600,0.620537,0.674503
200,0.500400,0.753936,0.634720
300,0.459700,0.605945,0.715190
400,0.461200,0.567429,0.710669
500,0.407000,0.564946,0.717902
600,0.257200,0.670025,0.727848


***** train metrics *****
  epoch                    =          5.0
  total_flos               = 1599648920GF
  train_loss               =       0.4514
  train_runtime            =   0:51:34.65
  train_samples_per_second =        7.162
  train_steps_per_second   =        0.225


NameError: ignored

In [ ]:
ViT_mura_sub_trainer.save_metrics("train", mura_subset_train_results.metrics)
ViT_mura_sub_trainer.save_state()

In [ ]:
print(mura_subset_train_results)

TrainOutput(global_step=695, training_loss=0.4513631055680968, metrics={'train_runtime': 3094.6552, 'train_samples_per_second': 7.162, 'train_steps_per_second': 0.225, 'total_flos': 1.7176099498074317e+18, 'train_loss': 0.4513631055680968, 'epoch': 5.0})


In [ ]:
metrics_mura_sub = ViT_mura_sub_trainer.evaluate(ViT_mura_test_subset)
ViT_mura_sub_trainer.log_metrics("eval", metrics_mura_sub)
ViT_mura_sub_trainer.save_metrics("eval", metrics_mura_sub)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.7179
  eval_loss               =     0.5649
  eval_runtime            = 0:00:57.08
  eval_samples_per_second =     19.375
  eval_steps_per_second   =      2.435


### All Data

In [ ]:
ViT_mura_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_mura = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura_Models/all",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
ViT_mura_trainer = Trainer(
    model=ViT_mura_model,
    args=training_args_mura,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_mura_train_all,
    eval_dataset=ViT_mura_test_all,
    tokenizer=feature_extractor,
)

In [ ]:
# run mura subset ViT model
mura_train_results = ViT_mura_trainer.train()
ViT_mura_trainer.save_model()
ViT_mura_trainer.log_metrics("train", mura_train_results.metrics)
ViT_mura_trainer.save_metrics("train", mura_train_results.metrics)
ViT_mura_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.670200,0.685423,0.545511
200,0.663400,0.676409,0.580231
300,0.651100,0.684792,0.561464
400,0.644600,0.660098,0.623710
500,0.666900,0.697544,0.533312
600,0.664500,0.666843,0.579606
700,0.661300,0.690251,0.560838
800,0.648100,0.646719,0.619331
900,0.605900,0.631545,0.636221
1000,0.615400,0.612960,0.654051


***** train metrics *****
  epoch                    =           5.0
  total_flos               = 13282174029GF
  train_loss               =        0.4554
  train_runtime            =    9:41:43.41
  train_samples_per_second =         5.273
  train_steps_per_second   =         0.165


In [ ]:
print(mura_train_results)

TrainOutput(global_step=5755, training_loss=0.4554387795210506, metrics={'train_runtime': 34903.4135, 'train_samples_per_second': 5.273, 'train_steps_per_second': 0.165, 'total_flos': 1.4261625768669512e+19, 'train_loss': 0.4554387795210506, 'epoch': 5.0})


In [ ]:
metrics_mura = ViT_mura_sub_trainer.evaluate(ViT_mura_test_all)
ViT_mura_trainer.log_metrics("eval", metrics_mura)
ViT_mura_trainer.save_metrics("eval", metrics_mura)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.6906
  eval_loss               =     0.6185
  eval_runtime            = 0:02:38.71
  eval_samples_per_second =     20.143
  eval_steps_per_second   =       2.52


In [ ]:
for val in val_sets:
    results = ViT_mura_sub_trainer.evaluate(val_sets[val])
    print(f'{val} : {results}')

shoulder : {'eval_loss': 0.7930088043212891, 'eval_accuracy': 0.5968028419182948, 'eval_runtime': 25.3953, 'eval_samples_per_second': 22.169, 'eval_steps_per_second': 2.796, 'epoch': 5.0}
humerus : {'eval_loss': 0.6127851605415344, 'eval_accuracy': 0.71875, 'eval_runtime': 13.8262, 'eval_samples_per_second': 20.83, 'eval_steps_per_second': 2.604, 'epoch': 5.0}
finger : {'eval_loss': 0.6994743943214417, 'eval_accuracy': 0.6334056399132321, 'eval_runtime': 20.2173, 'eval_samples_per_second': 22.802, 'eval_steps_per_second': 2.869, 'epoch': 5.0}
elbow : {'eval_loss': 0.45075997710227966, 'eval_accuracy': 0.7741935483870968, 'eval_runtime': 20.0666, 'eval_samples_per_second': 23.173, 'eval_steps_per_second': 2.94, 'epoch': 5.0}
wrist : {'eval_loss': 0.5498525500297546, 'eval_accuracy': 0.7450682852807283, 'eval_runtime': 28.4178, 'eval_samples_per_second': 23.19, 'eval_steps_per_second': 2.921, 'epoch': 5.0}


AttributeError: ignored

### Wrist

In [ ]:
#ViTForImageClassification

ViT_wrist_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_wrist = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/Wrist",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  gradient_accumulation_steps=4,
  metric_for_best_model='accuracy'
)

In [ ]:
#create trainer for wrist ViT model
ViT_wrist_trainer = Trainer(
    model=ViT_wrist_model,
    args=training_args_wrist,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_wrist_train_dataset,
    eval_dataset=ViT_wrist_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run wrist ViT model
#wrist_ViT_results = run_ViT(ViT_wrist_trainer)

wrist_train_results = ViT_wrist_trainer.train()
ViT_wrist_trainer.save_model()
ViT_wrist_trainer.log_metrics("train", wrist_train_results.metrics)
ViT_wrist_trainer.save_metrics("train", wrist_train_results.metrics)
ViT_wrist_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.370800,0.430953,0.804249
200,0.261500,0.493697,0.813354
300,0.154500,0.568769,0.811836


***** train metrics *****
  epoch                    =         4.98
  total_flos               = 3508041269GF
  train_loss               =       0.2595
  train_runtime            =   1:05:24.52
  train_samples_per_second =       12.424
  train_steps_per_second   =        0.097


In [ ]:
print(wrist_train_results)

TrainOutput(global_step=380, training_loss=0.25945617603628257, metrics={'train_runtime': 3924.5273, 'train_samples_per_second': 12.424, 'train_steps_per_second': 0.097, 'total_flos': 3.766730631186088e+18, 'train_loss': 0.25945617603628257, 'epoch': 4.98})


In [ ]:
metrics_wrist = ViT_wrist_trainer.evaluate(ViT_wrist_test_dataset)
ViT_wrist_trainer.log_metrics("eval", metrics_wrist)
ViT_wrist_trainer.save_metrics("eval", metrics_wrist)

***** eval metrics *****
  epoch                   =       4.98
  eval_accuracy           =     0.8134
  eval_loss               =     0.4937
  eval_runtime            = 0:00:53.99
  eval_samples_per_second =     12.205
  eval_steps_per_second   =      1.537


### Forearm

In [ ]:
ViT_forearm_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_forearm = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/forearm",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for forearm ViT model
ViT_forearm_trainer = Trainer(
    model=ViT_forearm_model,
    args=training_args_forearm,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_forearm_train_dataset,
    eval_dataset=ViT_forearm_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
forearm_train_results = ViT_forearm_trainer.train()
ViT_forearm_trainer.save_model()
ViT_forearm_trainer.log_metrics("train", forearm_train_results.metrics)
ViT_forearm_trainer.save_metrics("train", forearm_train_results.metrics)
ViT_forearm_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.528000,0.672646,0.654485
200,0.293400,0.624178,0.714286


***** train metrics *****
  epoch                    =         5.0
  total_flos               = 658551608GF
  train_loss               =      0.4539
  train_runtime            =  0:14:56.35
  train_samples_per_second =       10.18
  train_steps_per_second   =       0.324


In [ ]:
print(forearm_train_results)

TrainOutput(global_step=290, training_loss=0.4539256589166049, metrics={'train_runtime': 896.3537, 'train_samples_per_second': 10.18, 'train_steps_per_second': 0.324, 'total_flos': 7.07114405233152e+17, 'train_loss': 0.4539256589166049, 'epoch': 5.0})


In [ ]:
metrics_forearm = ViT_forearm_trainer.evaluate(ViT_forearm_test_dataset)
ViT_forearm_trainer.log_metrics("eval", metrics_forearm)
ViT_forearm_trainer.save_metrics("eval", metrics_forearm)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.7143
  eval_loss               =     0.6242
  eval_runtime            = 0:00:15.57
  eval_samples_per_second =     19.328
  eval_steps_per_second   =       2.44


### Hand

In [ ]:
ViT_hand_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_hand = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/Hand",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for hand ViT model
ViT_hand_trainer = Trainer(
    model=ViT_hand_model,
    args=training_args_hand,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_hand_train_dataset,
    eval_dataset=ViT_hand_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run hand ViT model
hand_train_results = ViT_hand_trainer.train()
ViT_hand_trainer.save_model()
ViT_hand_trainer.log_metrics("train", hand_train_results.metrics)
ViT_hand_trainer.save_metrics("train", hand_train_results.metrics)
ViT_hand_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.598700,0.699718,0.589130
200,0.605700,0.693682,0.589130
300,0.573600,0.746102,0.589130
400,0.587300,0.733341,0.589130
500,0.549300,0.715685,0.589130
600,0.580500,0.732235,0.589130
700,0.521200,0.760125,0.589130
800,0.509600,0.775598,0.593478


***** train metrics *****
  epoch                    =          5.0
  total_flos               = 2000192638GF
  train_loss               =       0.5589
  train_runtime            =   0:53:56.81
  train_samples_per_second =        8.562
  train_steps_per_second   =        0.269


In [ ]:
print(hand_train_results)

TrainOutput(global_step=870, training_loss=0.5588529784103919, metrics={'train_runtime': 3236.8138, 'train_samples_per_second': 8.562, 'train_steps_per_second': 0.269, 'total_flos': 2.1476904921684173e+18, 'train_loss': 0.5588529784103919, 'epoch': 5.0})


In [ ]:
metrics_hand = ViT_hand_trainer.evaluate(ViT_hand_test_dataset)
ViT_hand_trainer.log_metrics("eval", metrics_hand)
ViT_hand_trainer.save_metrics("eval", metrics_hand)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.5891
  eval_loss               =     0.6937
  eval_runtime            = 0:00:24.23
  eval_samples_per_second =     18.977
  eval_steps_per_second   =      2.393


### Humerus

In [ ]:
ViT_humerus_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_humerus = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/Humerus",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for humerus ViT model
ViT_humerus_trainer = Trainer(
    model=ViT_humerus_model,
    args=training_args_humerus,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_humerus_train_dataset,
    eval_dataset=ViT_humerus_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run humerus ViT model
humerus_train_results = ViT_humerus_trainer.train()
ViT_humerus_trainer.save_model()
ViT_humerus_trainer.log_metrics("train", humerus_train_results.metrics)
ViT_humerus_trainer.save_metrics("train", humerus_train_results.metrics)
ViT_humerus_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.233300,0.474974,0.833333
200,0.037400,0.593722,0.815972


***** train metrics *****
  epoch                    =         5.0
  total_flos               = 459001449GF
  train_loss               =      0.3019
  train_runtime            =  0:09:47.09
  train_samples_per_second =      10.833
  train_steps_per_second   =       0.341


In [ ]:
print(humerus_train_results)

TrainOutput(global_step=200, training_loss=0.3019163477420807, metrics={'train_runtime': 587.0984, 'train_samples_per_second': 10.833, 'train_steps_per_second': 0.341, 'total_flos': 4.928490539488051e+17, 'train_loss': 0.3019163477420807, 'epoch': 5.0})



Step	Training Loss	Validation Loss	Accuracy

100	0.329400	0.497600	0.802083

***** train metrics *****

  epoch                    =         4.0

  total_flos               = 369510601GF

  train_loss               =      0.4109

  train_runtime            =  0:31:41.39

  train_samples_per_second =       2.676

  train_steps_per_second   =       0.084

In [ ]:
metrics_humerus = ViT_humerus_trainer.evaluate(ViT_humerus_test_dataset)
ViT_humerus_trainer.log_metrics("eval", metrics_humerus)
ViT_humerus_trainer.save_metrics("eval", metrics_humerus)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.8333
  eval_loss               =      0.475
  eval_runtime            = 0:00:16.48
  eval_samples_per_second =     17.467
  eval_steps_per_second   =      2.183


### Elbow

In [ ]:
ViT_elbow_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_elbow = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/Elbow",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for elbow ViT model
ViT_elbow_trainer = Trainer(
    model=ViT_elbow_model,
    args=training_args_elbow,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_elbow_train_dataset,
    eval_dataset=ViT_elbow_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run humerus ViT model
elbow_train_results = ViT_elbow_trainer.train()
ViT_elbow_trainer.save_model()
ViT_elbow_trainer.log_metrics("train", elbow_train_results.metrics)
ViT_elbow_trainer.save_metrics("train", elbow_train_results.metrics)
ViT_elbow_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.644500,0.726395,0.554839
200,0.482000,0.498705,0.759140
300,0.514900,0.433372,0.815054
400,0.374300,0.561517,0.765591
500,0.353200,0.416712,0.815054
600,0.262300,0.472306,0.830108
700,0.243100,0.491573,0.830108


***** train metrics *****
  epoch                    =          5.0
  total_flos               = 1779352318GF
  train_loss               =       0.4291
  train_runtime            =   0:48:27.24
  train_samples_per_second =        8.481
  train_steps_per_second   =        0.267


In [ ]:
print(elbow_train_results)

TrainOutput(global_step=775, training_loss=0.4291260540100836, metrics={'train_runtime': 2907.2464, 'train_samples_per_second': 8.481, 'train_steps_per_second': 0.267, 'total_flos': 1.9105650039477658e+18, 'train_loss': 0.4291260540100836, 'epoch': 5.0})


In [ ]:
metrics_elbow = ViT_elbow_trainer.evaluate(ViT_elbow_test_dataset)
ViT_elbow_trainer.log_metrics("eval", metrics_elbow)
ViT_elbow_trainer.save_metrics("eval", metrics_elbow)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.8151
  eval_loss               =     0.4167
  eval_runtime            = 0:00:24.30
  eval_samples_per_second =     19.128
  eval_steps_per_second   =      2.427


Elbow 


Step	Training Loss	Validation Loss	Accuracy

100	0.633700	0.669449	0.593186

200	0.662600	0.674480	0.598662

300	0.684800	0.672798	0.593186

400	0.636100	0.645050	0.611438

500	0.732300	0.673190	0.595214

600	0.676000	0.656240	0.599473

700	0.645400	0.682207	0.600284

800	0.677900	0.636125	0.639424

900	0.618500	0.612148	0.653214

1000	0.594100	0.603044	0.662949

1100	0.613800	0.587694	0.680389

1200	0.547200	0.550778	0.715271

***** train metrics *****

  epoch                    =          4.0

  total_flos               = 1423481854GF

  train_loss               =       0.6496

  train_runtime            =   2:11:14.87

  train_samples_per_second =        2.505

  train_steps_per_second   =        0.157

### Finger

In [ ]:
ViT_finger_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_finger = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/Finger",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for finger ViT model
ViT_finger_trainer = Trainer(
    model=ViT_finger_model,
    args=training_args_finger,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_finger_train_dataset,
    eval_dataset=ViT_finger_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run finger ViT model
finger_train_results = ViT_finger_trainer.train()
ViT_finger_trainer.save_model()
ViT_finger_trainer.log_metrics("train", finger_train_results.metrics)
ViT_finger_trainer.save_metrics("train", finger_train_results.metrics)
ViT_finger_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.593800,0.600306,0.685466
200,0.431400,0.655124,0.639913
300,0.457900,0.686712,0.665944
400,0.427400,0.749594,0.646421
500,0.230100,0.878155,0.663774
600,0.242600,0.921132,0.646421
700,0.120400,1.174790,0.650759
800,0.125500,1.183439,0.655098


***** train metrics *****
  epoch                    =          5.0
  total_flos               = 1842501102GF
  train_loss               =        0.365
  train_runtime            =   0:51:32.60
  train_samples_per_second =        8.255
  train_steps_per_second   =        0.259


In [ ]:
print(finger_train_results)

TrainOutput(global_step=800, training_loss=0.36502562433481217, metrics={'train_runtime': 3092.6098, 'train_samples_per_second': 8.255, 'train_steps_per_second': 0.259, 'total_flos': 1.9783704948605338e+18, 'train_loss': 0.36502562433481217, 'epoch': 5.0})



Step	Training Loss	Validation Loss	Accuracy

100	0.639200	0.609000	0.678959

200	0.490100	0.633611	0.704989

300	0.510300	0.592520	0.685466

400	0.424900	0.702305	0.670282

500	0.290000	0.855674	0.633406

600	0.294100	0.870166	0.644252

***** train metrics *****

  epoch                    =          4.0

  total_flos               = 1474000882GF

  train_loss               =       0.4408

  train_runtime            =   1:51:04.16

  train_samples_per_second =        3.065

  train_steps_per_second   =        0.096

In [ ]:
metrics_finger = ViT_finger_trainer.evaluate(ViT_finger_test_dataset)
ViT_finger_trainer.log_metrics("eval", metrics_finger)
ViT_finger_trainer.save_metrics("eval", metrics_finger)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.6855
  eval_loss               =     0.6003
  eval_runtime            = 0:00:24.20
  eval_samples_per_second =     19.046
  eval_steps_per_second   =      2.396


### Shoulder

In [ ]:
ViT_shoulder_model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2,
    id2label=label_to_target_2,
    label2id=target_to_label_2
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args_shoulder = TrainingArguments(
  # save checkpoints and predictions to this directory
  output_dir="mura/Models/Shoulder",
  per_device_train_batch_size=vit_batch_size,
  evaluation_strategy="steps",
  num_train_epochs=vit_epochs,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=vit_lr,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
#create trainer for shoulder ViT model
ViT_shoulder_trainer = Trainer(
    model=ViT_shoulder_model,
    args=training_args_shoulder,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ViT_shoulder_train_dataset,
    eval_dataset=ViT_shoulder_test_dataset,
    tokenizer=feature_extractor,
)

In [ ]:
# run shoulder ViT model
shoulder_train_results = ViT_shoulder_trainer.train()
ViT_shoulder_trainer.save_model()
ViT_shoulder_trainer.log_metrics("train", shoulder_train_results.metrics)
ViT_shoulder_trainer.save_metrics("train", shoulder_train_results.metrics)
ViT_shoulder_trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.694700,0.693022,0.506217
200,0.693600,0.692978,0.506217
300,0.692700,0.691501,0.518650
400,0.701700,0.688578,0.513321
500,0.693200,0.691032,0.504440
600,0.690700,0.693723,0.515098
700,0.693400,0.687808,0.509769
800,0.678800,0.683712,0.525755
900,0.685000,0.682732,0.527531
1000,0.687600,0.685471,0.483126


***** train metrics *****
  epoch                    =          5.0
  total_flos               = 3023563795GF
  train_loss               =       0.6911
  train_runtime            =   1:32:41.84
  train_samples_per_second =        7.533
  train_steps_per_second   =        0.236


In [ ]:
print(shoulder_train_results)

TrainOutput(global_step=1310, training_loss=0.6910769983102347, metrics={'train_runtime': 5561.8463, 'train_samples_per_second': 7.533, 'train_steps_per_second': 0.236, 'total_flos': 3.246526904903332e+18, 'train_loss': 0.6910769983102347, 'epoch': 5.0})


In [ ]:
metrics_shoulder = ViT_shoulder_trainer.evaluate(ViT_shoulder_test_dataset)
ViT_shoulder_trainer.log_metrics("eval", metrics_shoulder)
ViT_shoulder_trainer.save_metrics("eval", metrics_shoulder)

***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.5293
  eval_loss               =     0.6814
  eval_runtime            = 0:00:28.21
  eval_samples_per_second =     19.952
  eval_steps_per_second   =      2.516
